# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
import os
import json

data_file = "/content/data"
data_file = "data"
train_file = os.path.join(data_file, 'train-claims.json')
dev_file = os.path.join(data_file, 'dev-claims.json')
evidence_file = os.path.join(data_file, 'evidence.json')
test_file = os.path.join(data_file, 'test-claims-unlabelled.json')

with open(train_file, 'r') as f:
    tr_claims = json.load(f)
tr_numbers = list(tr_claims.keys())
tr_texts = [tr_claims[claim_id]['claim_text'] for claim_id in tr_numbers]
claim_number_to_tr_id = {claim_id: i for i, claim_id in enumerate(tr_numbers)}

with open(dev_file, 'r') as f:
    dev_claims = json.load(f)
dev_numbers = list(dev_claims.keys())
dev_texts = [dev_claims[claim_id]['claim_text'] for claim_id in dev_numbers]

with open(evidence_file, 'r') as f:
    evidences = json.load(f)
evi_numbers = list(evidences.keys())
evidences_texts = [evidences[evidence_id] for evidence_id in evi_numbers]
evi_number_to_evi_id = {evi_number: i for i, evi_number in enumerate(evi_numbers)}

with open(test_file, 'r') as f:
    test_claims = json.load(f)
ts_numbers = list(test_claims.keys())
ts_texts = [test_claims[claim_id]['claim_text'] for claim_id in ts_numbers]

print("Train claims:", len(tr_claims))
print("Dev claims:", len(dev_claims))   
print("Evidences:", len(evidences))
print("Test claims:", len(test_claims))

Train claims: 1228
Dev claims: 154
Evidences: 1208827
Test claims: 153


In [32]:
# ####Local test#####
# import os
# import json

# train_file = "data/train-claims.json"
# with open(train_file, 'r') as f:
#     tr_claims = json.load(f)
    

# print("Train claims:", len(tr_claims))
# id = list(tr_claims.keys())[0]
# print("Train claim id:", id)
# tr_claim_test = tr_claims[id]
# tr_claim_test

In [4]:
######build vocab for train and evidence######
from collections import Counter
from nltk.tokenize import word_tokenize
from tqdm import tqdm 

def yield_tokens(data_iter):
    for text in data_iter:
        yield word_tokenize(text.lower())

# Build vocabulary manually using Counter
def build_vocab_from_iterator(token_iterator, min_freq, specials_tokens=('<cls>', '<unk>', '<pad>')):
    counter = Counter()
    for tokens in token_iterator:
        counter.update(tokens)

    vocab = {special_token: idx for idx, special_token in enumerate(specials_tokens)}
    current_index = len(specials_tokens)
    for token, freq in counter.items():
        if freq >= min_freq: # only include tokens with frequency >= min_freq
            vocab[token] = current_index
            current_index += 1

    stoi = vocab
    itos = {idx: token for token, idx in stoi.items()}
    return vocab, stoi, itos

# Create the vocab for train and evidence
# vocab, string_to_index, index_to_string = build_vocab_from_iterator(yield_tokens(tr_texts + evidences_texts), min_freq=5)
# print("Vocab size:", len(vocab))

# with open("vocab.json", "w") as f:
#     json.dump(vocab, f)

In [2]:
with open("vocab.json", "r") as f:
    vocab = json.load(f)
print("Vocab size:", len(vocab))

Vocab size: 112508


In [16]:
def text_to_indices(text, vocab):
    tokens = word_tokenize(text.lower())
    indices = [vocab.get(token, vocab["<unk>"]) for token in tokens]
    return [vocab["<cls>"]] + indices

tr_text_indices = [text_to_indices(text, vocab) for text in tr_texts]
dev_text_indices = [text_to_indices(text, vocab) for text in dev_texts]
ts_text_indices = [text_to_indices(text, vocab) for text in ts_texts]

# evidences_text_indices = [text_to_indices(text, vocab) for text in tqdm(evidences_texts)]

# with open("evidences_text_indices.json", "w") as f:
#     json.dump(evidences_text_indices, f)

100%|██████████| 1208827/1208827 [01:18<00:00, 15325.05it/s]


In [6]:
with open("evidences_text_indices.json", "r") as f:
    evidences_text_indices = json.load(f)
print("Evidences text indices:", len(evidences_text_indices))

Evidences text indices: 1208827


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')

lemmatizer = WordNetLemmatizer()
def word_tokenize_and_lemmatize(text):
    def lemmatize_word(token):
        token = lemmatizer.lemmatize(token, pos='v')
        token = lemmatizer.lemmatize(token, pos='n') if token != token else token
        return token
    
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    lemmed_tokens = [lemmatize_word(token) for token in tokens]
    return lemmed_tokens

[nltk_data] Downloading package stopwords to C:\Users\Salist
[nltk_data]     desk2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Salist
[nltk_data]     desk2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [34]:
# ####Local test#####
# word_tokenize_and_lemmatize(tr_claim_test['claim_text'])

In [35]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy import sparse
# import joblib

# evidence_ids = evidences.keys()
# evidence_text_list = [evidences[eid] for eid in evidence_ids]
# evidence_vectorizer = TfidfVectorizer(tokenizer=word_tokenize_and_lemmatize, max_features=5000)
# evidence_vectors = evidence_vectorizer.fit_transform(evidence_text_list)

# sparse.save_npz("evidence_vectors.npz", evidence_vectors)
# joblib.dump(evidence_vectorizer, "evidence_vectorizer.pkl")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import joblib

def save_tfidf_model(text_list, path_name):
    vectorizer = TfidfVectorizer(tokenizer=word_tokenize_and_lemmatize, max_features=5000)
    text_ids = list(text_list.keys())
    text_list = [text_list[id] for id in text_ids]

    tfidf_matrix = vectorizer.fit_transform(text_list)
    vector_path=str(path_name + "_vectors.npz")
    model_path=str(path_name + "_vectorizer.pkl")
    sparse.save_npz(vector_path, tfidf_matrix)
    joblib.dump(vectorizer, model_path)
    print(f"Saved TF-IDF vectors to '{vector_path}' and vectorizer to '{model_path}'")

In [ ]:
#######RUN ON COLAB##########
# save_tfidf_model(evidences, "evidences")

In [6]:
from scipy import sparse
import joblib

def load_tfidf(path_name):
    vector_path=str(path_name + "_vectors.npz")
    model_path=str(path_name + "_vectorizer.pkl")
    tfidf_matrix = sparse.load_npz(vector_path)
    vectorizer = joblib.load(model_path)
    print(f"Loaded TF-IDF matrix from '{vector_path}' and vectorizer from '{model_path}'")
    return tfidf_matrix, vectorizer

evidence_vectors, evidence_vectorizer = load_tfidf("evidences")

Loaded TF-IDF matrix from 'evidences_vectors.npz' and vectorizer from 'evidences_vectorizer.pkl'


In [ ]:
# from scipy import sparse
# import joblib
# # sparse.save_npz("evidence_vectors.npz", evidence_vectors)
# evidence_vectors = sparse.load_npz("evidence_vectors.npz")
# evidence_vectorizer = joblib.load("evidence_vectorizer.pkl")

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Retrival_Encode_LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=2, dropout=0.1):
        super(Retrival_Encode_LSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=vocab['<pad>'])
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, hidden_dim)

    def forward(self, x, real_len):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, real_len, batch_first=True, enforce_sorted=False)
        lstm_out, (lstm_hidden, _) = self.lstm(packed)
        hidden_forward = lstm_hidden[-2, :, :]
        hidden_backward = lstm_hidden[-1, :, :]
        hidden_cat = torch.cat((hidden_forward, hidden_backward), dim=1)
        output_embedding = self.fc(hidden_cat)
        return output_embedding


class Retrival_Encode_GRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=2, dropout=0.1):
        super(Retrival_Encode_GRU, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=vocab['<pad>'])
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim) #bidirectional

    def forward(self, x, real_len):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, real_len, batch_first=True, enforce_sorted=False)
        output, hidden = self.gru(packed)
        hidden_forward = hidden[-2, :, :]
        hidden_backward = hidden[-1, :, :]
        hidden_cat = torch.cat((hidden_forward, hidden_backward), dim=1)
        output_embedding = self.fc(hidden_cat)
        return output_embedding

#test generate_batched_embeddings
def generate_batched_embeddings_ts(model, evidences_text_indices, batch_size=32):
    all_embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(evidences_text_indices), batch_size)):
            batch_evi_indices = evidences_text_indices[i:i + batch_size]
            real_len = [len(indices) for indices in batch_evi_indices]
            batch_evi_indices_pad = torch.nn.utils.rnn.pad_sequence([torch.tensor(indices) for indices in batch_evi_indices], batch_first=True, padding_value=vocab['<pad>'])
            # batch_embeddings = model(evidence_batch_indices.cuda(), real_len)
            batch_embeddings = model(batch_evi_indices_pad, real_len)

            all_embeddings.extend(batch_embeddings.tolist())
            del batch_embeddings
            torch.cuda.empty_cache()

    return torch.tensor(all_embeddings)

#returns f1 for each claim
def evaluate_retrival(claims, top_evidence_id):
    claims_f1 = []
    for i, claim_id in enumerate(claims.keys()):
        correct = 0
        recall = 0.0
        precision = 0.0
        fscore = 0.0
        
        claim = claims[claim_id]
        true_evidence_ids = claim['evidences']
        for true_evidence in true_evidence_ids:
            true_evidence_id = int(true_evidence.split('-')[1])
            if true_evidence_id in top_evidence_id:
                correct += 1
        if correct > 0:
            recall = correct / len(true_evidence_ids)
            precision = correct / len(top_evidence_id[i])
            fscore = (2 * precision * recall) / (precision + recall)
        claims_f1.append(fscore)
    return claims_f1

#returns avg f1 for input claims
def calc_f1(claim_texts_indices, claims, evidences_text_indices, model, bathch_size=32):
    model.eval()

    claim_embeddings = generate_batched_embeddings_ts(model, claim_texts_indices, batch_size=bathch_size)
    claim_embeddings_norm = F.normalize(claim_embeddings, p=2, dim=1)

    evidences_embeddings = generate_batched_embeddings_ts(model, evidences_text_indices, batch_size=bathch_size)
    evidences_embeddings_norm = F.normalize(evidences_embeddings, p=2, dim=1)

    cos_similarities = torch.matmul(claim_embeddings_norm, evidences_embeddings_norm.T)

    top_k = 5
    top_k_indices = torch.topk(cos_similarities, top_k, dim=1).indices
    top_k_indices = top_k_indices.numpy()

    claims_f1 = evaluate_retrival(claims, top_k_indices)
    model.train()
    return np.mean(claims_f1)


In [21]:
#train generate_text_embeddings
def generate_text_embeddings_tr(model, text_indices):  
  real_len = [len(indices) for indices in text_indices]
  pad_text_indices = torch.nn.utils.rnn.pad_sequence(
      [torch.tensor(indices) for indices in text_indices],
        batch_first=True, padding_value=vocab['<pad>'])

#   embeddings = model(pad_text_indices.cuda(), real_len)
  embeddings = model(pad_text_indices, real_len)
  embeddings_norm = F.normalize(embeddings, p=2, dim=1)
  return embeddings_norm

#returns all evidence indices included in the claim tr
def get_real_evidences(batch_claims):
    evi_indices = []
    claims_pos_evidence_indices = []
    for claim in batch_claims:
        pos_evi_indices = []
        for evi_number in claim["evidences"]:
            cur_evi_index = evi_number_to_evi_id[evi_number]
            if cur_evi_index not in evi_indices:
                evi_indices.append(cur_evi_index)
            pos_evi_indices.append(evi_indices.index(cur_evi_index))
        claims_pos_evidence_indices.append(pos_evi_indices)
    return evi_indices, claims_pos_evidence_indices

#loss based on cosine similarity of pos and neg evidence
def contrastive_loss(claim_embedding, pos_evi_embeddings, neg_evi_embeddings, temperature=0.1):
    """Compute contrastive loss for a claim"""
    pos_sim = torch.exp(torch.matmul(claim_embedding, pos_evi_embeddings.T) / temperature).sum()
    neg_sim = torch.exp(torch.matmul(claim_embedding, neg_evi_embeddings.T) / temperature).sum()

    loss = -torch.log(pos_sim / (pos_sim + neg_sim))
    return loss.mean()

In [1]:
#########RUN ON COLAB##########
########Training Model##########
import random
max_epochs = 10  ######Change Later######
max_steps = 100  ######Change Later######
batch_size = 64
embedding_dim = 128
hidden_dim = 128
num_layers = 2
dropout = 0.2
learning_rate = 0.001
test_period = 40
log_period = 20

model= Retrival_Encode_GRU(len(vocab), embedding_dim, hidden_dim, num_layers, dropout)
model = model
# model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

step = 0
model.train()
max_f1 = 0.0
for epoch in range(max_epochs):
    if step > max_steps:
        break
    print(f"Epoch {epoch+1}/{max_epochs}")
    random.shuffle(tr_numbers)

    for i in range(0, len(tr_numbers), batch_size):
        if step > max_steps:
            break
        step += 1
        if step % 10 == 0:
            print(f"Step {step}")
        batch_claim_ids = tr_numbers[i:i + batch_size] # claim ids
        batch_claims = [tr_claims[claim_id] for claim_id in batch_claim_ids]
        batch_tr_indices = [claim_number_to_tr_id[claim_id] for claim_id in batch_claim_ids] #claims' train indices

        claim_text_indices = [tr_text_indices[i] for i in batch_tr_indices]
        norm_claim_embeddings = generate_text_embeddings_tr(model, claim_text_indices)

        ##Todo modify positive and negative evidence list##
        all_real_evi, pos_evi = get_real_evidences(batch_claims)
        cur_evi_indices = [evidences_text_indices[id] for id in all_real_evi]
        norm_evi_embeddings = generate_text_embeddings_tr(model, cur_evi_indices)

        loss = []
        for i, claim_embedding in enumerate(norm_claim_embeddings):
            pos_evi_embedding = norm_evi_embeddings[torch.tensor(pos_evi[i])]
            # neg_evidences = [j for j in range(len(all_real_evi)) if j not in pos_evi[i]] #some postive evidence of oter claims
            # neg_evi_embedding = norm_evi_embeddings[torch.tensor(neg_evidences)]
            neg_evi_embedding = norm_evi_embeddings[torch.tensor([j for j in range(len(all_real_evi)) if j not in pos_evi[i]])]

            loss.append(contrastive_loss(claim_embedding, pos_evi_embedding, neg_evi_embedding))
        
        loss = torch.mean(torch.stack(loss))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        if step % log_period == 0:
            print(f"Step {step}, Loss: {loss.item()}")
        if step % test_period == 0:
            print("Evaluating on dev set...")
            dev_f1 = calc_f1(dev_text_indices, dev_claims, evidences_text_indices, model)
            avg_f1 = np.mean(dev_f1)
            print(f"Avg F1 on dev set: {avg_f1}")
            if avg_f1 > max_f1:
                max_f1 = avg_f1
                print(f"New best F1: {max_f1}, model saved.")
                torch.save(model.state_dict(), "best_model.pth")
            

NameError: name 'Retrival_Encode_GRU' is not defined

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

evidences_vectors, evidences_vectorizer = load_tfidf("evidences")
dev_ids = list(dev_claims.keys())
dev_claims_vectors = evidences_vectorizer.transform([dev_claims[id]['claim_text'] for id in dev_ids])

def top_k_evidence(vectors, k=5):
    similarity = cosine_similarity(vectors, evidences_vectors)
    top_k_indices = np.argsort(similarity, axis=1)[:, -k:]
    top_k_indices = np.flip(top_k_indices, axis=1)
    return top_k_indices

dev_top5_evidence_id = top_k_evidence(dev_claims_vectors, k=5)

NameError: name 'load_tfidf' is not defined

In [ ]:
def evaluate_retrival(claims, top_evidence_id):
    evidence_f1 = []
    for i, claim_id in enumerate(claims.keys()):
        correct = 0
        recall = 0.0
        precision = 0.0
        fscore = 0.0
        
        claim = claims[claim_id]
        true_evidence_ids = claim['evidences']
        for true_evidence in true_evidence_ids:
            true_evidence_id = int(true_evidence.split('-')[1])
            if true_evidence_id in top_evidence_id:
                correct += 1
        if correct > 0:
            recall = correct / len(true_evidence_ids)
            precision = correct / len(top_evidence_id[i])
            fscore = (2 * precision * recall) / (precision + recall)
        evidence_f1.append(fscore)
    return evidence_f1

dev_top5_evidence_f1 = evaluate_retrival(dev_claims, dev_top5_evidence_id)
print("Dev claims F1 score:", np.mean(dev_top5_evidence_f1))

Dev claims F1 score: 0.08017419088847662
